## Import des librairies

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, accuracy_score, auc, roc_curve, confusion_matrix
from xgboost import XGBClassifier

In [2]:
# Commenter l'une ou l'autre lignes suivant que l'on souhaite travailler sur le dataset enrichi ou non. 

data=pd.read_csv('weatherAUS.csv')
#data=pd.read_csv('enricheredaus_sanscluster.csv')
#On supprime du Dataframe les lignes où RainTomorrow est nul
liste=data.index[data['RainTomorrow'].isnull()]
data.drop(liste , inplace=True)

In [3]:
data.head(5)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [4]:
#On convertit le type de la variable Date
data['Date']=data['Date'].astype('datetime64')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142193 entries, 0 to 145458
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           142193 non-null  datetime64[ns]
 1   Location       142193 non-null  object        
 2   MinTemp        141556 non-null  float64       
 3   MaxTemp        141871 non-null  float64       
 4   Rainfall       140787 non-null  float64       
 5   Evaporation    81350 non-null   float64       
 6   Sunshine       74377 non-null   float64       
 7   WindGustDir    132863 non-null  object        
 8   WindGustSpeed  132923 non-null  float64       
 9   WindDir9am     132180 non-null  object        
 10  WindDir3pm     138415 non-null  object        
 11  WindSpeed9am   140845 non-null  float64       
 12  WindSpeed3pm   139563 non-null  float64       
 13  Humidity9am    140419 non-null  float64       
 14  Humidity3pm    138583 non-null  float64       
 15  

In [6]:
la = LabelEncoder()
data['RainToday'] = la.fit_transform(data['RainToday'])
data['RainTomorrow'] = la.fit_transform(data['RainTomorrow'])

# Data cleaning

In [7]:
data = pd.get_dummies(data)

In [8]:
data = data.dropna()

# Entraînement du modèle

In [9]:
cor = data.corr()
cor2 = data.corr()
cor = np.where(cor <0,cor*-1 , cor)
cor=pd.DataFrame(cor)
cor.columns=cor2.columns
cor.index =cor2.index 

/tmp/ipykernel_439/3685902465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  cor = data.corr()
/tmp/ipykernel_439/3685902465.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  cor2 = data.corr()


In [10]:
cor

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
MinTemp,1.000000,0.749940,0.115248,0.509595,0.075757,0.133227,0.134638,0.150001,0.182607,0.072295,...,0.027205,0.031179,0.051786,0.057723,0.000336,0.066333,0.076941,0.076885,0.000066,0.083743
MaxTemp,0.749940,1.000000,0.065842,0.653135,0.464779,0.050740,0.012703,0.028445,0.500402,0.447072,...,0.022876,0.010766,0.092010,0.018796,0.026041,0.086555,0.070401,0.072535,0.003407,0.073867
Rainfall,0.115248,0.065842,1.000000,0.073939,0.243855,0.105338,0.050920,0.043308,0.258623,0.276014,...,0.019762,0.001404,0.016354,0.012456,0.020704,0.018552,0.003677,0.024548,0.015288,0.017705
Evaporation,0.509595,0.653135,0.073939,1.000000,0.370672,0.221870,0.208450,0.138085,0.557311,0.420953,...,0.030505,0.029643,0.019301,0.031307,0.019459,0.014295,0.004228,0.041805,0.037521,0.019612
Sunshine,0.075757,0.464779,0.243855,0.370672,1.000000,0.039792,0.001752,0.044286,0.502676,0.631821,...,0.017141,0.012975,0.041561,0.014479,0.023737,0.022974,0.020499,0.022935,0.013082,0.001499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WindDir3pm_SSW,0.066333,0.086555,0.018552,0.014295,0.022974,0.021132,0.015799,0.030046,0.044545,0.027533,...,0.058049,0.057854,0.071869,0.072071,0.064221,1.000000,0.071159,0.070034,0.062821,0.069517
WindDir3pm_SW,0.076941,0.070401,0.003677,0.004228,0.020499,0.002490,0.022080,0.003049,0.005283,0.011325,...,0.062616,0.062406,0.077523,0.077741,0.069273,0.071159,1.000000,0.075543,0.067764,0.074987
WindDir3pm_W,0.076885,0.072535,0.024548,0.041805,0.022935,0.081361,0.021772,0.065648,0.012053,0.006249,...,0.061625,0.061418,0.076297,0.076511,0.068177,0.070034,0.075543,1.000000,0.066692,0.073800
WindDir3pm_WNW,0.000066,0.003407,0.015288,0.037521,0.013082,0.078289,0.011200,0.059424,0.009249,0.014013,...,0.055279,0.055093,0.068440,0.068632,0.061156,0.062821,0.067764,0.066692,1.000000,0.066200


In [11]:
x=data

In [12]:
x = data
y = data['RainTomorrow']
x.drop('RainTomorrow', inplace = True, axis = 1)
x.drop('qtyraintomorrow', inplace = True, axis = 1,errors='ignore')
#x.drop('qtyraintomorrow', inplace = True, axis = 1)
x.drop('Date', inplace = True, axis = 1)
x

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
6049,17.9,35.2,0.0,12.0,12.3,48.0,6.0,20.0,20.0,13.0,...,0,0,0,0,0,0,1,0,0,0
6050,18.4,28.9,0.0,14.8,13.0,37.0,19.0,19.0,30.0,8.0,...,0,0,0,0,1,0,0,0,0,0
6052,19.4,37.6,0.0,10.8,10.6,46.0,30.0,15.0,42.0,22.0,...,1,0,0,0,0,0,0,0,0,0
6053,21.9,38.4,0.0,11.4,12.2,31.0,6.0,6.0,37.0,22.0,...,0,0,0,0,0,0,0,0,0,1
6054,24.2,41.0,0.0,11.2,8.4,35.0,17.0,13.0,19.0,15.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142298,19.3,33.4,0.0,6.0,11.0,35.0,9.0,20.0,63.0,32.0,...,0,0,0,0,0,0,0,0,0,0
142299,21.2,32.6,0.0,7.6,8.6,37.0,13.0,11.0,56.0,28.0,...,0,0,0,1,0,0,0,0,0,0
142300,20.7,32.8,0.0,5.6,11.0,33.0,17.0,11.0,46.0,23.0,...,0,0,0,0,0,0,0,1,0,0
142301,19.5,31.8,0.0,6.2,10.6,26.0,9.0,17.0,62.0,58.0,...,1,0,0,0,0,0,0,0,0,0


In [13]:
y.value_counts()

0    45361
1    12729
Name: RainTomorrow, dtype: int64

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

## Interpolation des valeur manquantes

# Evaluation du modèle

In [15]:
rf = RandomForestClassifier(n_estimators = 200)
rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=200)

In [16]:
predicted = rf.predict(x_test)
print("La précision est de {} %".format(accuracy_score(predicted, y_test)*100))

La précision est de 86.71888448958512 %


In [17]:
pd.crosstab(predicted, y_test)

RainTomorrow,0,1
row_0,,
0,8725,1145
1,398,1350


In [18]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, x, y, cv=5)
print(scores)


[0.80134274 0.84162506 0.7130315  0.84455156 0.85255638]
